In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("miadul/brain-tumor-dataset")

print("Path to dataset files:", path)

100%|██████████| 852k/852k [00:01<00:00, 806kB/s]

Extracting files...
Path to dataset files: C:\Users\Lenovo\.cache\kagglehub\datasets\miadul\brain-tumor-dataset\versions\1


### Librerias

In [1]:
from scipy import stats
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

In [2]:
df = pd.read_csv("brain_tumor_dataset.csv")
df.head()

,Patient_ID,Age,Gender,Tumor_Type,Tumor_Size,Location,Histology,Stage,Symptom_1,Symptom_2,Symptom_3,Radiation_Treatment,Surgery_Performed,Chemotherapy,Survival_Rate,Tumor_Growth_Rate,Family_History,MRI_Result,Follow_Up_Required
0,1,73,Male,Malignant,5.375612,Temporal,Astrocytoma,III,Vision Issues,Seizures,Seizures,No,No,No,51.312579,0.111876,No,Positive,Yes
1,2,26,Male,Benign,4.847098,Parietal,Glioblastoma,II,Headache,Headache,Nausea,Yes,Yes,Yes,46.373273,2.165736,Yes,Positive,Yes
2,3,31,Male,Benign,5.588391,Parietal,Meningioma,I,Vision Issues,Headache,Seizures,No,No,No,47.072221,1.884228,No,Negative,No
3,4,29,Male,Malignant,1.436600,Temporal,Medulloblastoma,IV,Vision Issues,Seizures,Headache,Yes,No,Yes,51.853634,1.283342,Yes,Negative,No
4,5,54,Female,Benign,2.417506,Parietal,Glioblastoma,I,Headache,Headache,Seizures,No,No,Yes,54.708987,2.069477,No,Positive,Yes


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Patient_ID           20000 non-null  int64  
 1   Age                  20000 non-null  int64  
 2   Gender               20000 non-null  object 
 3   Tumor_Type           20000 non-null  object 
 4   Tumor_Size           20000 non-null  float64
 5   Location             20000 non-null  object 
 6   Histology            20000 non-null  object 
 7   Stage                20000 non-null  object 
 8   Symptom_1            20000 non-null  object 
 9   Symptom_2            20000 non-null  object 
 10  Symptom_3            20000 non-null  object 
 11  Radiation_Treatment  20000 non-null  object 
 12  Surgery_Performed    20000 non-null  object 
 13  Chemotherapy         20000 non-null  object 
 14  Survival_Rate        20000 non-null  float64
 15  Tumor_Growth_Rate    20000 non-null 

In [4]:
df.describe()

,Patient_ID,Age,Tumor_Size,Survival_Rate,Tumor_Growth_Rate
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,10000.500000,49.433700,5.236552,70.131699,1.547701
std,5773.647028,17.389296,2.754359,17.270834,0.835995
min,1.000000,20.000000,0.500410,40.002386,0.100017
25%,5000.750000,34.000000,2.836686,55.224439,0.830433
50%,10000.500000,49.000000,5.200675,70.305066,1.541821
75%,15000.250000,65.000000,7.624299,85.043395,2.262484
max,20000.000000,79.000000,9.999420,99.998125,2.999835


In [5]:
# Revisar cuántos valores únicos tiene cada columna categórica
cat_cols = df.select_dtypes(exclude=["number"]).columns.tolist()

for col in cat_cols:
    if col != "Tumor_Type":
        unicos = df[col].nunique()
        print(f"{col}: {unicos} valores únicos")

Gender: 2 valores únicos
Location: 4 valores únicos
Histology: 4 valores únicos
Stage: 4 valores únicos
Symptom_1: 4 valores únicos
Symptom_2: 4 valores únicos
Symptom_3: 4 valores únicos
Radiation_Treatment: 2 valores únicos
Surgery_Performed: 2 valores únicos
Chemotherapy: 2 valores únicos
Family_History: 2 valores únicos
MRI_Result: 2 valores únicos
Follow_Up_Required: 2 valores únicos


In [6]:
# Quitamos las columnas Symptom_1, Symptom_2 y Symptom_3
cols_a_quitar = ["Symptom_1", "Symptom_2", "Symptom_3"]

df_reducido = df.drop(columns=cols_a_quitar)
print("Se eliminaron las columnas:", cols_a_quitar)
print("Nueva forma del dataset:", df_reducido.shape)


Se eliminaron las columnas: ['Symptom_1', 'Symptom_2', 'Symptom_3']
Nueva forma del dataset: (20000, 16)


In [7]:
#  Separamos X (todo menos Tumor_Type) y y (solo Tumor_Type)
X = df_reducido.drop(columns=["Tumor_Type"])
y = df_reducido["Tumor_Type"]

In [8]:
#  Convertimos texto en números (porque el modelo no entiende strings)
X = pd.get_dummies(X, drop_first=True)   # convierte categóricas en columnas 0/1
y = (y == "Malignant").astype(int)       # 1 si es maligno, 0 si es benigno


In [9]:
#  Escalamos los valores (importante para SVC)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [10]:
#  Creamos el modelo SVC lineal (sin optimizar todavía)
modelo = SVC(kernel="linear", C=1, probability=True, random_state=42)


In [ ]:
#  Evaluamos con validación cruzada (10 folds)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(modelo, X_scaled, y, cv=kfold, scoring="roc_auc")

print("AUC promedio:", round(scores.mean(), 4))
print("AUC por fold:", np.round(scores, 4))

## Optimización bayesiana de C

In [ ]:
from skopt import BayesSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold

In [ ]:
#  Definimos el modelo (igual que antes, kernel lineal)
modelo = SVC(kernel="linear", probability=True, random_state=42)

In [ ]:
#  Rango de búsqueda para C (en escala logarítmica)
parametros = {"C": (1e-4, 1e3, "log-uniform")}

In [ ]:
#  Validación cruzada
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [ ]:
#  Configuramos la búsqueda bayesiana
busqueda = BayesSearchCV(
    estimator=modelo,
    search_spaces=parametros,
    n_iter=20,            # número de pruebas que hará (puedes bajarlo a 10 si quieres más rápido)
    cv=cv,
    scoring="roc_auc",    # métrica a optimizar
    random_state=42,
    n_jobs=-1
)

In [ ]:
#  Ejecutamos la búsqueda
print("Buscando el mejor valor de C, esto puede tardar un poco...")
busqueda.fit(X_scaled, y)

In [ ]:
#  Resultados
print("\n✅ Optimización terminada:")
print("Mejor C encontrado:", busqueda.best_params_["C"])
print("AUC promedio (10 folds):", round(busqueda.best_score_, 4))